In [15]:
import pandas as pd
import pickle

In [16]:
df_products = pd.read_json('../../../data/interim/final/products.json.gz', orient="records", compression="gzip")
df_reviews = pd.read_json('../../../data/interim/final/reviews.json.gz', orient="records", compression="gzip")

In [17]:
model = pickle.load(open('../../../models/item_based_collaborative_filtering/knn.pkl', 'rb'))

In [18]:
# Get all items that have at least 1 rating and sorted by similarity
def find_similar_items(product_id, df_products):
    neighbors = model.get_neighbors(model.trainset.to_inner_iid(product_id), k=model.trainset.n_items)
    df_1 = pd.DataFrame({'product_id' : [model.trainset.to_raw_iid(inner_id) for inner_id in neighbors]})
    return df_1.merge(df_products, how='inner', on='product_id')

# Get unrated items for a user
def get_user_unrated_items(user_id, df_products, df_reviews):
    df_user_reviews = df_reviews[df_reviews['user_id'] == user_id]
    return df_products[~df_products['product_id'].isin(df_user_reviews['product_id'])]

def predict_rating(user_id, product_id):
    return model.predict(user_id, product_id)

def recommend(user_id, product_id, n = 10):
    df_similar_items = find_similar_items(product_id, df_products)
    df_user_unrated_items = get_user_unrated_items(user_id, df_similar_items, df_reviews)
    
    if n < len(df_user_unrated_items):
        df_user_unrated_items = df_user_unrated_items[:n]
    
    df_user_unrated_items['pre'] = df_user_unrated_items['product_id'].apply(lambda id: predict_rating(user_id, id).est)
    df_user_unrated_items = df_user_unrated_items.sort_values(by=['pre'], ascending=False)
    return df_user_unrated_items

In [19]:
user_id = 'A0203183BAH3TR08FZGB'
product_id = 'B0043T7FHK'

In [20]:
df_products[df_products['product_id'] == product_id][['product_id', 'name']].head()

,product_id,name
226,B0043T7FHK,ASUS VE248H 24&quot; Full HD 1920x1080 2ms HDM...


In [21]:
recommend(user_id, product_id)[['product_id', 'name', 'pre']]

,product_id,name,pre
0,B015WCV70W,HP Pavilion 21.5-Inch IPS LED HDMI VGA Monitor...,5.0
2,B009A5204K,LG Tone HBS-730 Wireless Stereo Headset - Black,5.0
3,B00906GBBC,ASUS PB238Q 23&quot; Full HD 1920x1080 IPS Dis...,5.0
4,B00OL0L1VM,Sceptre E Series E275W-1920 V1 27&quot; Screen...,5.0
5,B003LPTAYI,Sennheiser HD 202 II Professional Headphones (...,5.0
6,B0149QBOF0,Dell Gaming S2716DGR 27.0&quot; Screen LED-Lit...,5.0
7,B0058UUR6E,ASUS VS248H-P 24&quot; Full HD 1920x1080 2ms H...,5.0
9,B00EZSUWFG,ViewSonic VX2252MH 22 Inch 2ms 75Hz 1080p Gami...,5.0
1,B0043T7FKC,ASUS VK278Q 27&quot; Full HD 1920x1080 2ms HDM...,5.0
8,B00B5Q6Y8U,ASUS VS207D-P 19.5&quot; HD+ 1600x900 VGA Back...,5.0
